In [23]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
%matplotlib inline
import random
import matplotlib.pyplot as plt
from tensorflow.keras import layers as layers
from tensorflow.keras import models as models

In [24]:
(X_train_raw, y_train_raw), (X_test_raw, y_test_raw) = tf.keras.datasets.mnist.load_data()

In [25]:
#
# Helper function to show a list of images with their relating titles
#
def show_images(images, title_texts):
    cols = 5
    rows = int(len(images)/cols) + 1
    plt.figure(figsize=(30,20))
    index = 1    
    for x in zip(images, title_texts):        
        image = x[0]        
        title_text = x[1]
        plt.subplot(rows, cols, index)        
        plt.imshow(image, cmap=plt.cm.gray)
        if (title_text != ''):
            plt.title(title_text, fontsize = 15);        
        index += 1


In [ ]:
images_2_show = []
titles_2_show = []
for i in range(0, 10):
    r = random.randint(1, 60000)
    images_2_show.append(X_train_raw[r])
    titles_2_show.append('training image [' + str(r) + '] = ' + str(y_train_raw[r]))    

for i in range(0, 5):
    r = random.randint(1, 10000)
    images_2_show.append(X_train_raw[r])        
    titles_2_show.append('test image [' + str(r) + '] = ' + str(y_train_raw[r]))    

show_images(images_2_show, titles_2_show)

In [27]:
print('X_train_raw shape: ', X_train_raw.shape)
print('y_train shape: ', y_train_raw.shape)
print('x_test shape: ', X_test_raw.shape)
print('y_test shape: ', y_test_raw.shape)
print('\n')
print('data about X_train_raw: ')
print('min: ', X_train_raw.min())
print('max: ', X_train_raw.max())
print('mean: ', X_train_raw.mean())

X_train_raw shape:  (60000, 28, 28)
y_train shape:  (60000,)
x_test shape:  (10000, 28, 28)
y_test shape:  (10000,)


data about X_train_raw: 
min:  0
max:  255
mean:  33.318421449829934


In [28]:
X_train = np.reshape(X_train_raw, (X_train_raw.shape[0], X_train_raw.shape[1], X_train_raw.shape[2], 1)).astype('float32') / 255.0
y_train = tf.keras.utils.to_categorical(y_train_raw, num_classes=10) #np.reshape(y_train, (60000, 1)) #
X_test = np.reshape(X_test_raw, (X_test_raw.shape[0], X_test_raw.shape[1], X_test_raw.shape[2], 1)).astype('float32') / 255.0
y_test = tf.keras.utils.to_categorical(y_test_raw, num_classes=10)

print('X_train after reshape: ', X_train.shape)
print('y_train after reshape: ', y_train.shape)
print('X_test after reshape: ', X_test.shape)
print('y_test after reshape: ', y_test.shape)
print('\n')
print('data about X_train: ')
print('min: ', X_train.min())
print('max: ', X_train.max())
print('mean: ', X_train.mean())

X_train after reshape:  (60000, 28, 28, 1)
y_train after reshape:  (60000, 10)
X_test after reshape:  (10000, 28, 28, 1)
y_test after reshape:  (10000, 10)


data about X_train: 
min:  0.0
max:  1.0
mean:  0.13066062


In [29]:
model = models.Sequential([
    layers.Conv2D(filters=4, kernel_size=(3,3), padding='valid', activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Conv2D(filters=4,kernel_size=(3,3), padding='valid', activation='relu'),
    layers.MaxPool2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(units=64,activation='relu'),
    layers.Dense(units=10, activation='softmax')
])

In [30]:
model.compile(optimizer='adam',
              loss = 'categorical_crossentropy',
              metrics = ['accuracy'], run_eagerly=True)

Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op _EagerConst in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op Fill in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op VarHandleOp in device /job:localhost/replica:0/task:0/device:GPU:0
Executing op AssignVariableOp in device /job:localhost/replica:0/task:0/device:GPU:0


In [ ]:

history = model.fit(X_train, y_train, batch_size=1024, epochs=4)

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc}")

313/313 [==============================] - 2s 6ms/step - loss: 0.1072 - accuracy: 0.9648
Test Accuracy: 0.9648000001907349


In [ ]:
start = 20
length = 20

predictions = model.predict(X_test[start:start+length])
print("Predicted labels:", np.argmax(predictions, axis=1))
print("True labels:     ", np.argmax(y_test[start:start+length], axis=1))

1/1 [==============================] - 0s 16ms/step
Predicted labels: [9 6 6 5 4 0 7 4 0 1 3 1 3 6 7 2 7 1 2 1]
True labels:      [9 6 6 5 4 0 7 4 0 1 3 1 3 4 7 2 7 1 2 1]


In [ ]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_2 (Conv2D)           (None, 26, 26, 4)         40        
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 13, 13, 4)         0         
 g2D)                                                            
                                                                 
 conv2d_3 (Conv2D)           (None, 11, 11, 4)         148       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 5, 5, 4)           0         
 g2D)                                                            
                                                                 
 flatten_1 (Flatten)         (None, 100)               0         
                                                                 
 dense_2 (Dense)             (None, 64)               

In [ ]:
# Check if TensorFlow is built with GPU support
print("Is TensorFlow built with GPU support? ", tf.test.is_built_with_cuda())

# List available devices
print("Available devices:")
for device in tf.config.list_physical_devices():
    print(device)

Is TensorFlow built with GPU support?  False
Available devices:
PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')


In [ ]:
import tensorflow as tf; print(tf.config.list_physical_devices('GPU'))

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


I0000 00:00:1723682482.877122    2323 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723682482.975239    2323 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1723682482.975281    2323 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:07:00.0/numa_node
Your kernel may have been built without NUMA support.


In [ ]:
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]